In [ ]:
!pip install -U torchtext==0.8.0

     |████████████████████████████████| 6.9 MB 4.3 MB/s 


In [ ]:
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.legacy.data import Field,BucketIterator, TabularDataset
from torch.nn.utils.rnn import pad_sequence
from nltk.translate.bleu_score import SmoothingFunction
from torchtext.datasets import Multi30k

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
 

import nltk
import os
import re
import random
import math
import time
import spacy
from google.colab import drive

drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [ ]:
MAXLENGTH = 10
BATCHSIZE = 128


train_path = 'drive/My Drive/CMPUT499/seq2seq/data_30k_re.csv'
test_path = 'drive/My Drive/CMPUT499/seq2seq/data_test_30k_re.csv'
valid_path = 'drive/My Drive/CMPUT499/seq2seq/data_valid_30k_re.csv'
SRC = Field(lower=True, 
            init_token="<sos>", 
            eos_token="<eos>",
            fix_length=MAXLENGTH,
            include_lengths=True)

TRG = Field(lower=True, 
            init_token="<sos>", 
            eos_token="<eos>",
            fix_length=MAXLENGTH)


train_data = TabularDataset(train_path, 
                      format="csv", 
                      skip_header=True,
                      fields=[("src", SRC), ("trg", TRG)])

valid_data = TabularDataset(valid_path, 
                      format="csv", 
                      skip_header=True,
                      fields=[("src", SRC), ("trg", TRG)])

test_data = TabularDataset(test_path, 
                      format="csv", 
                      skip_header=True,
                      fields=[("src", SRC), ("trg", TRG)])
print(len(train_data))

In [ ]:
SRC.build_vocab(train_data)
TRG.build_vocab(train_data)
print(SRC.vocab.stoi)
print(TRG.vocab.stoi)

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
            (train_data, valid_data, test_data), 
              batch_size = BATCHSIZE,
              sort_within_batch = True,
              sort_key=lambda x: len(x.src),
              sort=True,
              device = device)



In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers)
        
        # self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        #src = [src len, batch size]
        
        embedded = self.embedding(src)
        
        
        #embedded = [src len, batch size, emb dim]
                
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.cpu())
                
        packed_outputs, hidden = self.rnn(packed_embedded)
                                 
        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
        # print("outputs", outputs)
        # print("hidden", hidden)
        # print("hidden", hidden)
            
        # outputs= self.rnn(embedded)
        # print(outputs)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        # self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.embedding(input)
        
        #embedded = [1, batch size, emb dim]
                
        output, hidden = self.rnn(embedded, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        # print(prediction)
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden = self.encoder(src, src_len)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden = self.decoder(input, hidden)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
# size of vocabulary
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)

ENC_EMB_DIM = 512
DEC_EMB_DIM = 512
# ENC_HID_DIM = 15
# DEC_HID_DIM = 15
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0
DEC_DROPOUT = 0

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS)

model = Seq2Seq(enc, dec, device).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src, src_len = batch.src
        trg = batch.trg
        # print("src.shape", src.shape)
        optimizer.zero_grad()
        
        output = model(src, src_len, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_len = batch.src
            trg = batch.trg

            output = model(src, src_len, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item() # add loss iteratively from batch
        
    return epoch_loss / len(iterator)

In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = MAXLENGTH):

    model.eval()
        
    tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    # print("token with eos, sos", tokens)
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    # print("token encoded", src_indexes)
    
    src_indexes += [src_field.vocab.stoi[src_field.pad_token] for i in range(max_len-len(src_indexes))] 

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)
    # print("token tensor", src_indexes)

    with torch.no_grad():
        hidden = model.encoder(src_tensor, src_len)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden = model.decoder(trg_tensor, hidden)

        pred_token = output.argmax(1).item()
        
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
        trg_indexes.append(pred_token)
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:]

# model.load_state_dict(torch.load('tut1-model.pt'))

# src = ['a', 'b', 'c', 'd']
# translation = translate_sentence(src, SRC, TRG, model, device)

# print('predicted trg = {', translation,'}')

In [ ]:
N_EPOCHS = 50
CLIP = 1

best_bleu = float('-inf')

train_loss_lst = []
valid_loss_lst = []
train_bleu_lst = []
test_bleu_lst = []
train_exact_match_lst = []
test_exact_match_lst = []


for epoch in range(N_EPOCHS):
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    train_loss_lst.append(train_loss)
    valid_loss_lst.append(valid_loss)

    train_BLEU = 0
    train_exact_match = 0
    for i in range(len(train_data)):
      src = train_data[i].src
      # print(f'src = {src}')

      translation = translate_sentence(src, SRC, TRG, model, device)
      trg = train_data[i].trg
      # print(f'trg = {trg}')
      # print(f'translation = {translation}')

      train_BLEU += float(nltk.translate.bleu_score.modified_precision([trg], translation, n=1))
      # print(train_BLEU/(i+1))
      if trg == translation:
        train_exact_match += 1
        
    train_BLEU /= len(train_data)
    train_exact_match /= len(train_data)
    train_bleu_lst.append(train_BLEU)
    train_exact_match_lst.append(train_exact_match)


    test_BLEU = 0
    test_exact_match = 0

    for i in range(len(test_data)):
      src = test_data[i].src
      # print(f'src = {src}')

      translation = translate_sentence(src, SRC, TRG, model, device)
      trg = test_data[i].trg
      # print(f'trg = {trg}')
      # print(f'translation = {translation}')

      test_BLEU += float(nltk.translate.bleu_score.modified_precision([trg], translation, n=1))

      if trg == translation:
        test_exact_match += 1

    test_BLEU /= len(test_data)
    test_exact_match /= len(test_data)
    test_bleu_lst.append(test_BLEU)
    test_exact_match_lst.append(test_exact_match)

    
    print(f'| Train BLEU: {train_BLEU:.3f} | Exact Match Accuracy: {train_exact_match:.3f} |')

  
    if train_exact_match > best_bleu:
        best_bleu = train_exact_match
        torch.save(model.state_dict(), 'tut1-model.pt')
        print('model saved in', epoch)

    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

# plotting result
epochs = range(1, N_EPOCHS + 1)
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel('index of Epoch')
ax1.set_ylabel('Train Exact Match', color=color)
l1 = ax1.plot(epochs, train_exact_match_lst, color=color, label= "Train Exact Match")
# ax1.legend(loc="upper right")
ax1.tick_params(axis='y', labelcolor=color)

colorSet = ["blue", "orange", "purple", "green", "gray"]

i = 0
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
# color = 'tab:blue'
ax2.get_shared_y_axes().join(ax1, ax2)
color = 'tab:'+colorSet[i]
ax2.set_ylabel('Test Exact Match', color=color)  # we already handled the x-label with ax1
l3 = ax2.plot(epochs, test_exact_match_lst, color=color, label = "Test Exact Match")
ax2.tick_params(axis='y', labelcolor=color)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


| Train BLEU: 0.809 | Exact Match Accuracy: 0.079 |
model saved in 0
	Train Loss: 1.807 | Train PPL:   6.095
	 Val. Loss: 1.239 |  Val. PPL:   3.454
| Train BLEU: 0.999 | Exact Match Accuracy: 0.960 |
model saved in 1
	Train Loss: 0.612 | Train PPL:   1.844
	 Val. Loss: 0.203 |  Val. PPL:   1.225
| Train BLEU: 1.000 | Exact Match Accuracy: 0.998 |
model saved in 2
	Train Loss: 0.089 | Train PPL:   1.093
	 Val. Loss: 0.045 |  Val. PPL:   1.046
| Train BLEU: 1.000 | Exact Match Accuracy: 1.000 |
model saved in 3
	Train Loss: 0.029 | Train PPL:   1.029
	 Val. Loss: 0.021 |  Val. PPL:   1.021
| Train BLEU: 1.000 | Exact Match Accuracy: 1.000 |
model saved in 4
	Train Loss: 0.015 | Train PPL:   1.015
	 Val. Loss: 0.012 |  Val. PPL:   1.012
| Train BLEU: 1.000 | Exact Match Accuracy: 1.000 |
model saved in 5
	Train Loss: 0.009 | Train PPL:   1.009
	 Val. Loss: 0.008 |  Val. PPL:   1.008
| Train BLEU: 1.000 | Exact Match Accuracy: 1.000 |
model saved in 6
	Train Loss: 0.006 | Train PPL:   1.0

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

In [ ]:
BLEU = 0
exact_match = 0
print(len(train_data))
for i in range(0, len(train_data)):
  src = train_data[i].src
  # print(f'src = {src}')

  translation = translate_sentence(src, SRC, TRG, model, device)
  trg = train_data[i].trg
  # print(f'trg = {trg}')
  # print(f'translation = {translation}')

  BLEU += float(nltk.translate.bleu_score.modified_precision([trg], translation, n=1))
  
  if trg == translation:
    exact_match += 1

BLEU /= len(train_data)
exact_match /= len(train_data)
print(f'| Train BLEU: {BLEU:.3f} | Exact Match Accuracy: {exact_match:.3f} |')


BLEU = 0
exact_match = 0
for i in range(0, len(test_data)):
  src = test_data[i].src
  # print(f'src = {src}')

  translation = translate_sentence(src, SRC, TRG, model, device)
  trg = test_data[i].trg
  # print(f'trg = {trg}')
  # print(f'translation = {translation}')
  BLEU += float(nltk.translate.bleu_score.modified_precision([trg], translation, n=1))

  if trg == translation:
    exact_match += 1

BLEU /= len(test_data)
exact_match /= len(test_data)
print(f'| Test BLEU: {BLEU:.3f} | Exact Match Accuracy: {exact_match:.3f} |')


In [ ]:
src = ['a', 'b', 'c', 'd']
translation = translate_sentence(src, SRC, TRG, model, device)

print('predicted trg = {', translation,'}')